Here we present some tools which were used preparing the experiments.

In [ ]:
!pip install Biopython
!pip install gdown
!pip install pydna

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

In [ ]:
from Bio import SeqIO
import numpy as np
import matplotlib.pyplot as plt
import json
import gdown
import gc
from pydna.assembly import Assembly
from pydna.dseqrecord import Dseqrecord
from pydna.design import primer_design, assembly_fragments
%matplotlib inline

In [ ]:
url = 'https://drive.google.com/uc?id=1P-dkZg7Uy0NPdzUwY-a-qpTsNfw4FURX'
output = 'salmonella-enterica.reads.fna'
gdown.download(url, output, quiet=False)

url = 'https://drive.google.com/uc?id=1Or3tTlBb98A8u84XaCEpZPiK-IOYjzgh'
output = 'salmonella-enterica-variant.reads.fna'
gdown.download(url, output, quiet=False)

In [ ]:
def build_kmers(sequence, dict_kmers, ksize):
    """
    Build k-mers from sequence
    """
    n_kmers = len(sequence) - ksize + 1
    for i in range(n_kmers):
        kmer = sequence[i:i + ksize]
        if kmer not in dict_kmers:
            dict_kmers[kmer] = 0
        dict_kmers[kmer] += 1
    return

In [ ]:
def check_kmer(path, KSIZE):
  dict_kmers = {}
  fasta_sequences = SeqIO.parse(open(path), 'fasta')
  for fasta in fasta_sequences:
    build_kmers(str(fasta.seq), dict_kmers, KSIZE)
  return dict_kmers

In [ ]:
# Missing kmers in original
kmers_freq = [v for k, v in kmers_dict.items() if k not in kmers_dict_var]
# Missing kmers in variant
kmers_freq_var = [v for k, v in kmers_dict_var.items() if k not in kmers_dict]

In [ ]:
kmers_assembly = [Dseqrecord(k) for k, f in kmers_dict.items() if ((f > 42.5) & (f < 50) & (k not in kmers_dict_var))]

In [ ]:
seq = Assembly(kmers_assembly, limit = 22).assemble_linear()
print(seq[0].seq.watson)

In [ ]:
kmers_assembly_var = [Dseqrecord(k) for k, f in kmers_dict_var.items() if ((f > 40) & (f < 50) & (k not in kmers_dict))]

In [ ]:
seq_var = Assembly(kmers_assembly_var,limit = 22).assemble_linear()
print(seq_var[0].seq.watson)

In [ ]:
kmers_assembly = [Dseqrecord(k) for k, f in kmers_dict.items() if ((f > 40) & (f < 42.5) & (k not in kmers_dict_var))]

In [ ]:
seq = Assembly(kmers_assembly,limit = 22).assemble_linear()
print(seq[0].seq.watson)

In [ ]:
kmers_assembly_var = [Dseqrecord(k) for k, f in kmers_dict_var.items() if ((f > 30) & (f < 35) & (k not in kmers_dict))]

In [ ]:
seq_var = Assembly(kmers_assembly_var,limit = 22).assemble_linear()
print(seq_var[0].seq.watson)